In [3]:
import spacy
import re
import pandas as pd
import numpy as np

nlp = spacy.load("en_core_web_sm")

ESSENTIAL_WORDS = {"avocado", "all", "baking"}
ESSENTIAL_ADJECTIVES = {"olive", "canola", "coconut", "sesame", "almond", "hot", "rosemary", "green", "red", "yellow", "black", "zest"}
ESSENTIAL_ADV = {"baking", "all", "rising"}  
EXCLUDED_WORDS = {"taste", "needed", "optional", "divided", "degrees", "f", "c", "halves", "inch", "pieces"}

# Aquí indicamos qué sustantivos consideramos "compound" 
# cuando van precedidos por un adjetivo o un compound. 
# Así "chili powder" o "garlic powder" se mantienen juntos.
TARGET_NOUNS = {"powder", "oil", "sauce", "milk", "paste", "meal", "flour"}

def clean_ingredient_with_pos(text: str) -> str:
    """
    1) Separa guiones ("freeze-dried" -> "freeze dried") para tokenizar mejor.
    2) Usa spaCy para POS tagging + dependencias.
    3) Combina ADJ/compound + NOUN cuando NOUN está en TARGET_NOUNS (por ej. "chili powder").
    4) Elimina palabras EXCLUDED, etc.
    5) Lemmatiza sustantivos para unificar singular/plural ("eggs" -> "egg").
    6) Reconstruye en un único string limpio.
    """
    text = text.replace("-", " ")
    doc = nlp(text)

    processed = set()  # para marcar tokens ya procesados
    keep_tokens = []

    for token in doc:
        # Si ya lo hemos procesado en una combinación anterior, lo saltamos
        if token.i in processed:
            continue

        lemma = token.lemma_.lower()  # para comparaciones en minúscula

        # 1) Si está en la lista EXCLUDED, ignoramos este token
        if lemma in EXCLUDED_WORDS:
            continue

        # 2) Detectar si este token es un adjetivo (ADJ) o compound (dep_) 
        #    que modifica a un sustantivo "target" (e.g. powder)
        #    - token.dep_ puede ser 'amod' (adjectival modifier) o 'compound'
        #    - token.head es el NOUN al que modifica
        if token.dep_ in ("amod", "compound") and token.head.pos_ == "NOUN":
            head_lemma = token.head.lemma_.lower()
            if head_lemma in TARGET_NOUNS:
                # => Combinamos p.ej. "chili" + "powder" => "chili powder"
                combined = f"{token.lemma_.lower()} {head_lemma}"
                keep_tokens.append(combined)
                
                # Marcamos ambos tokens como procesados
                processed.add(token.i)
                processed.add(token.head.i)
                continue

        # 3) Si es un NOUN o PROPN, lo mantenemos (lematizado) 
        #    a menos que se haya procesado en la combinación anterior
        if token.pos_ in ["NOUN", "PROPN"] and token.i not in processed:
            # Lematizamos para unificar singular/plural
            keep_tokens.append(token.lemma_)

        # 4) Si es un ADJ y está en ESSENTIAL_ADJECTIVES, lo guardamos
        elif token.pos_ == "ADJ":
            if lemma in ESSENTIAL_ADJECTIVES:
                keep_tokens.append(token.lemma_)

        # 5) Si es un ADV y está en ESSENTIAL_ADV, lo guardamos
        elif token.pos_ == "ADV":
            if lemma in ESSENTIAL_ADV:
                keep_tokens.append(token.lemma_)

        # 6) Si es una palabra "esencial" por su lemma (e.g. "avocado"), la guardamos
        elif lemma in ESSENTIAL_WORDS:
            keep_tokens.append(token.lemma_)

    # Reconstruimos y limpiamos espacios, etc.
    cleaned = " ".join(keep_tokens)
    cleaned = re.sub(r"\s+", " ", cleaned).strip(",. ")

    return cleaned

In [4]:
recipes = pd.read_json(r"..\recetas_scrapper\recetas_allrecipes.jsonl", lines = True)

In [5]:
ing_list = [r["nombre"] for r in recipes.loc[74:, "ingredientes"]]

In [6]:
all_ing_list = [item for sublist in ing_list for item in sublist]

In [7]:
from tqdm import tqdm
clean_ing_list = []
for item in all_ing_list:
    clean_ing = clean_ingredient_with_pos(item)
    clean_ing_list.append(clean_ing)

    print(f"{item} -> {clean_ing}")

cabbage, thinly sliced -> cabbage
white sugar -> sugar
bacon -> bacon
corn bread mix -> corn bread mix
cayenne pepper -> cayenne pepper
chopped onion -> onion
fresh jalapeño peppers, seeded and julienned -> jalapeño pepper
salt and pepper to taste -> salt pepper
hard-cooked eggs, chopped -> egg
avocados, peeled and chopped -> avocado
seeded and chopped tomatoes -> tomato
diced red onion -> red onion
salt and ground black pepper to taste -> salt ground black pepper
mayonnaise -> mayonnaise
sour cream -> cream
lemon juice -> lemon juice
hot sauce (such as Frank's RedHot) -> hot sauce Frank RedHot
dried great Northern beans, soaked overnight -> bean
canola oil -> canola oil
onion, chopped -> onion
bacon, diced -> bacon
garlic, minced -> garlic
dried thyme -> thyme
red pepper flakes -> red pepper flake
pure maple syrup -> maple syrup
tomato puree -> tomato puree
Worcestershire sauce -> worcestershire sauce
mustard powder -> mustard powder
ham bone with some meat -> ham bone meat
boiling wa

In [8]:
len(clean_ing_list)

6562

In [9]:
pd.DataFrame(clean_ing_list).value_counts().tail(60)

0                                                     
vegetable bouillon base Better Bouillon                   1
vegetable corn pea carrot green bean lima bean            1
vegetarian bean                                           1
tuna                                                      1
turkey broth                                              1
turkey ham ham                                            1
turkey liver                                              1
turkey wing                                               1
turnip                                                    1
unbleached flour                                          1
unbleached flour bread flour                              1
vanilla bean                                              1
warm milk                                                 1
wasabi paste                                              1
water amount                                              1
water bean                                   

In [10]:
clean_ing_list = list(set(clean_ing_list))

In [11]:
len(clean_ing_list)

1182

In [12]:
import spacy
import re

# Carga del modelo de spaCy para español
nlp_es = spacy.load("es_core_news_sm")

# Palabras “esenciales” (en lemma) que siempre queremos mantener
ESSENTIAL_WORDS = {"aguacate", "canela", "albahaca", "sódico", "sodio"}  # ajusta según tu dominio

# Adjetivos esenciales que queremos preservar (en lemma).
# Ej.: “integral”, “fresco”, “verde”, etc.
ESSENTIAL_ADJECTIVES = {
    "integral", "fresco", "verde", "rojo", "amarillo", "negro",
    "picante", "canola", "coco", "almendra", "rallado"
}

# Adverbios que, si aparecen, deseamos conservar (casos muy concretos).
ESSENTIAL_ADV = {"todo", "rising"}  # Ajusta según tu dominio

# Palabras que queremos excluir siempre (por ejemplo, “opcional”, “necesario”, etc.)
EXCLUDED_WORDS = {
    "opcional", "necesario", "grados", "f", "c", "gusto","pellizco", "pizca","cucharada","cucharadita",
    "taza","vaso","litro","mililitro","gramo","kilogramo","onza","gota","chorrito","puñado","unidad","lata",
    "tableta","trozo","diente","ramita","manojo","pieza","bolsa","copa", "m", "l", "xl","s", "temperatura", "ambiente",
    "caliente", "fría", "frío"}

# SUSTANTIVOS en español que queremos tratar como un “compuesto”
# cuando vayan precedidos por un adjetivo o compound (borrowed) 
# Ej.: “polvo” (powder), “aceite” (oil), “salsa” (sauce), “harina” (flour), etc.
TARGET_NOUNS = {
    "polvo",   # powder
    "aceite",  # oil
    "salsa",   # sauce
    "leche",   # milk
    "pasta",   # paste
    "harina"   # flour
}

def clean_ingredient_with_pos_es(text: str) -> str:
    """
    1) Reemplaza guiones "freeze-dried" -> "freeze dried"
    2) Analiza con POS tagging + dependencias usando spaCy (es_core_news_sm)
    3) Combina ADJ/compound + NOUN si el NOUN está en TARGET_NOUNS
    4) Elimina palabras en EXCLUDED_WORDS
    5) Lematiza sustantivos para unificar sing/plural
    6) Retorna un string limpio
    """
    # Quitar guiones
    text = text.replace("-", " ")
    
    text = re.sub(r"\(.*?\)", "", text)  # Eliminar paréntesis y su contenido

    doc = nlp_es(text)
    
    processed = set()  # Para no procesar 2 veces el mismo token
    keep_tokens = []

    for token in doc:
        # Si ya está procesado en un paso anterior, lo ignoramos
        if token.i in processed:
            continue
        
        lemma = token.lemma_.lower()  # lemma en minúsculas
        lower = token.text.lower()  # texto en minúsculas

        # 1) Ignorar palabras excluidas (p. ej. "opcional", "gusto", etc.)
        if lemma in EXCLUDED_WORDS:
            continue

        if lower in ESSENTIAL_WORDS:
            keep_tokens.append(lower)

        # 2) Revisar si es un adjetivo (ADJ) o compound (dep_) que modifica un NOUN
        #    En español, 'compound' es menos frecuente, pero a veces aparece
        #    en términos importados o expresiones compuestas.
        if token.dep_ in ("amod", "compound") and token.head.pos_ == "NOUN":
            head_lemma = token.head.lemma_.lower()
            # Si el sustantivo principal está en TARGET_NOUNS (p. ej. 'polvo')
            if head_lemma in TARGET_NOUNS:
                # Combinamos: "chili" + "powder" => "chili powder"
                # En español podría ser "ajo" + "polvo" => "ajo polvo"
                combined = f"{lemma} {head_lemma}"
                keep_tokens.append(combined)
                
                # Marcamos como procesados (para no añadirlos de nuevo)
                processed.add(token.i)
                processed.add(token.head.i)
                continue

        # 3) Si es un sustantivo (NOUN) o nombre propio (PROPN) (no procesado antes):
        if token.pos_ in ["NOUN", "PROPN"] and token.i not in processed:
            keep_tokens.append(lemma)  # lematizado

        # 4) Si es un adjetivo (ADJ) y está en la lista de esenciales, lo conservamos
        elif token.pos_ == "ADJ":
            if lemma in ESSENTIAL_ADJECTIVES:
                keep_tokens.append(lemma)

        # 5) Si es un adverbio (ADV) y está en ESSENTIAL_ADV, lo conservamos
        elif token.pos_ == "ADV":
            if lemma in ESSENTIAL_ADV:
                keep_tokens.append(lemma)        

    # Reconstruimos en un solo string
    cleaned = " ".join(keep_tokens)
    # Limpiamos espacios, comas, puntos sobrantes
    cleaned = re.sub(r"\s+", " ", cleaned).strip(",. ")

    return cleaned

In [13]:
clean_ingredient_with_pos_es("Canela molida")

'canela'

In [14]:
recetas = pd.read_json(r"recetas_scrapper\recetas_dap_no_delay.jsonl", lines = True)
ing_lista = [r["nombre"] for r in recetas.loc[74:, "ingredientes"]]
all_ing_lista = [item for sublist in ing_lista for item in sublist]

C:\Users\yanru\AppData\Local\Temp\ipykernel_2616\681471977.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  recetas = pd.read_json(r"recetas_scrapper\recetas_dap_no_delay.jsonl", lines = True)


ValueError: Expected object or value

In [75]:
clean_ing_lista = []
for item in all_ing_lista:
    clean_ing = clean_ingredient_with_pos_es(item)
    clean_ing_lista.append(clean_ing)

    print(f"{item} -> {clean_ing}")

Cebolla -> cebolla
Berenjena -> berenjena
Pimiento rojo morrón -> pimiento rojo morrón
Pimiento verde grande -> pimiento verde
Tomates en conserva natural -> tomates conserva
Aceite de oliva virgen extra -> aceite virgen
Bicarbonato sódico o azúcar -> bicarbonato sódico azúcar
Sal necesaria -> sal
Leche -> leche
Clara de huevo Lonchas -> clara huevo lonchas
Harina de repostería -> harina repostería
Azúcar -> azúcar
Mantequilla -> mantequilla
Levadura química , 4 cucharaditas -> levadura
Sal , un pellizco -> sal
Extracto de vainilla -> extracto vainilla
Harina de maíz amarilla (no maizena ni precocida ni polenta) -> harina maíz amarillo
Nueces (peso ya peladas y molidas) -> nueces
Manteca de cerdo a temperatura ambiente -> manteca cerdo
Azúcar glasé y más para espolvorear -> azúcar glasé
Canela molida -> canela
Rape fresco 2 colas medianas aproximadamente 500 g -> rape fresco cola
Langostinos crudos -> langostino
Huevo -> huevo
Pan rallado una cucharada sopera -> pan rallado
Cebolla -> 

In [53]:
for i in nlp_es("Canela molida"):
    print(f"{i.text}->{i.pos_}")

Canela->VERB
molida->ADJ


In [78]:
pd.DataFrame(clean_ing_lista).value_counts().head(60)

0                 
sal                   386
aceite virgen         240
azúcar                207
pimienta negro        164
huevo                 149
ajo                   126
mantequilla           121
cebolla               108
                      105
leche                  92
agua                   81
harina trigo           73
tomate                 73
levadura               68
zumo limón             59
vino                   53
pimentón               48
harina repostería      45
queso                  44
pimiento verde         42
zanahoria              42
huevos                 40
patata                 37
esencia vainilla       36
miel                   35
cebolleta              33
aceite girasol         33
ralladura limón        32
orégano                31
pimiento rojo          30
canela canela          29
comino                 29
perejil fresco         29
almendra               28
calabacín              27
azúcar glasé           27
limón                  27
nuez               

In [77]:
set(clean_ing_lista)

{'',
 'sardinilla parrocha aceite',
 'romero',
 'clavo',
 'laurel',
 'ali oli',
 'endibias',
 'tocino ternilla',
 'panecillo',
 'carga sifón',
 'pan día',
 'jamón york taquito',
 'aceitunas verde hueso generoso',
 'harina trigo mezcla',
 'melocotón',
 'salmón',
 'yogur',
 'amaretto licor',
 'salsa',
 'caracoles',
 'aguacate aguacate tira',
 'caramelo',
 'cebolla',
 'salmón lomo filete',
 'papada cerdo piel',
 'mejillón escabeche',
 'vino vino',
 'pan hogaza torrija rebanada',
 'ajo marinada',
 'berberechos fresco',
 'lacón fresco',
 'canela canela',
 'ralladura postre tartar',
 'bacon',
 'cerdo',
 'lima tira piel',
 'bacon loncha',
 'muslo pollo piel hueso',
 'pan molde rebanada',
 'chile pasta tartar',
 'boniato',
 'maizena',
 'melón',
 'hojas menta fresco',
 'salchichas fresco',
 'salmón crudo tira corta',
 'copo coco',
 'fraiche crema fresco',
 'tortas',
 'indicación',
 'azúcar vainilla',
 'arándano fresco congelado',
 'guindilla',
 'mermelada frambuesa cereza',
 'vacío',
 'zumo lim